# Docker:

```bash

docker run --rm -d -v $(pwd):/datos --name jupyterlab_django -p 8000:8000 -p 8989:8888 palmoreck/django:3.0.0

```

# Example for some satellital data

From:

https://realpython.com/get-started-with-django-1/#blog-app-models

and:

https://docs.djangoproject.com/en/3.1/topics/db/examples/many_to_many/

In [1]:
%%bash
name_django_proj=mydjangoproj

cd /datos/example-django/
mkdir myproj
cd /datos/example-django/myproj
/home/miuser/.local/bin/django-admin startproject $name_django_proj

mv $name_django_proj/manage.py ./
mv $name_django_proj/$name_django_proj/* $name_django_proj
rm -r $name_django_proj/$name_django_proj/

python3 manage.py migrate #if run it will create db.sqlite3 file


Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

In [2]:
%%bash
cd /datos/example-django/myproj
python3 manage.py startapp satellital_images

In [3]:
%%bash
file=/datos/example-django/myproj/mydjangoproj/settings.py
sed -i "/'django.contrib.staticfiles',/a \ \ \ \ 'satellital_images'," $file

In [4]:
%%file /datos/example-django/myproj/satellital_images/models.py
from django.db import models

class Collection(models.Model):
    collection_type = models.CharField(unique=True, max_length=5)

class Sensor(models.Model):
    sensor_type = models.CharField(unique=True, max_length=10)
    collection_att = models.ManyToManyField("Collection")

class PathRow(models.Model):
    path_row_att = models.IntegerField(unique=True)
    
class Image(models.Model):
    download_url = models.CharField(max_length=200, default=None)
    sensor_image_att = models.ForeignKey(Sensor, related_name="sensor_images", on_delete=models.CASCADE)
    path_row_image_att = models.ForeignKey(PathRow, related_name="pathrow_images", on_delete=models.CASCADE)
    collection_image_att = models.ForeignKey(Collection, related_name="collection_images", on_delete=models.CASCADE)
    acquisition_date = models.DateTimeField()
    

Overwriting /datos/example-django/myproj/satellital_images/models.py


In [5]:
%%bash

cd /datos/example-django/myproj

python3 manage.py makemigrations 

#python3 manage.py makemigrations satellital_images

Migrations for 'satellital_images':
  satellital_images/migrations/0001_initial.py
    - Create model Collection
    - Create model PathRow
    - Create model Sensor
    - Create model Image


In [6]:
%%bash

cd /datos/example-django/myproj/

python3 manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, satellital_images, sessions
Running migrations:
  Applying satellital_images.0001_initial... OK


**Cmd to see query in sql: python3 manage.py sqlmigrate satellital_images 0001_initial.py**

In [7]:
import os
os.chdir("/datos/example-django/myproj")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mydjangoproj.settings")
import django
django.setup()
from satellital_images.models import Collection, PathRow, Sensor

In [8]:
scene_0 = {"acquisitionDate": "2013-03-09", 
           "startTime": "2013-03-09", 
           "endTime": "2013-03-09", 
           "lowerLeftCoordinate": {"latitude": 18.09711, "longitude": -100.57469}, 
           "upperLeftCoordinate": {"latitude": 19.81465, "longitude": -100.19015}, 
           "upperRightCoordinate": {"latitude": 19.47153, "longitude": -98.52673}, 
           "lowerRightCoordinate": {"latitude": 17.75737, "longitude": -98.92825}, 
           "spatialFootprint": {"type": "Polygon", 
                                "coordinates": [[[-100.57469, 18.09711], 
                                                 [-98.92825, 17.75737], 
                                                 [-98.52673, 19.47153], 
                                                 [-100.19015, 19.81465], 
                                                 [-100.57469, 18.09711]]]}, 
           "sceneBounds": "-100.57469,17.75737,-98.52673,19.81465", 
           "browseUrl": "https://ims.cr.usgs.gov/browse/landsat_8_c1/2013/026/047/LT08_L1GT_026047_20130309_20170505_01_T2.jpg", 
           "dataAccessUrl": "https://earthexplorer.usgs.gov/metadata/full/landsat_8_c1/LT80260472013068LGN02/", 
           "downloadUrl": "https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260472013068LGN02/EE/", 
           "entityId": "LT80260472013068LGN02", 
           "displayId": "LT08_L1GT_026047_20130309_20170505_01_T2", 
           "metadataUrl": "https://earthexplorer.usgs.gov/metadata/full/landsat_8_c1/LT80260472013068LGN02/?responseType=viewXml", 
           "fgdcMetadataUrl": "https://earthexplorer.usgs.gov/metadata/fgdc/landsat_8_c1/LT80260472013068LGN02/", 
           "modifiedDate": "2013-03-09 17:01:29", 
           "orderUrl": "null", 
           "bulkOrdered": "false", 
           "ordered": "false", 
           "summary": ""}

In [9]:
download_url = scene_0["downloadUrl"]
d = scene_0["displayId"]
list_d = d.split("_")
sensor=list_d[0]
path_row = list_d[2]
collection=list_d[5]
acq_date = scene_0["acquisitionDate"]

In [10]:
coll = Collection(collection_type=collection)
pr = PathRow(path_row_att=path_row)
sen = Sensor(sensor_type=sensor)

In [11]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
coll.save()
pr.save()
sen.save()
sen.collection_att.add(coll)

In [12]:
from satellital_images.models import Image

In [13]:
img = Image(download_url=download_url,
            sensor_image_att=sen,
            path_row_image_att=pr,
            collection_image_att=coll,
            acquisition_date=acq_date)

In [14]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
img.save()

/home/miuser/.local/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1370: RuntimeWarning: DateTimeField Image.acquisition_date received a naive datetime (2013-03-09 00:00:00) while time zone support is active.
  RuntimeWarning)


In [15]:
import sqlite3

connection = sqlite3.connect("db.sqlite3")
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_user',), ('django_session',), ('satellital_images_collection',), ('satellital_images_pathrow',), ('satellital_images_sensor',), ('satellital_images_sensor_collection_att',), ('satellital_images_image',)]


**Next just to check, normally is not necessary to execute:**

In [16]:
rows = cursor.execute("SELECT * FROM satellital_images_image").fetchall()
print(rows)

[(1, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260472013068LGN02/EE/', '2013-03-09 00:00:00', 1, 1, 1)]


In [17]:
rows = cursor.execute("SELECT * FROM satellital_images_collection").fetchall()
print(rows)

[(1, '01')]


In [18]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor").fetchall()
print(rows)

[(1, 'LT08')]


In [19]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor_collection_att").fetchall()
print(rows)

[(1, 1, 1)]


In [20]:
rows = cursor.execute("SELECT * FROM satellital_images_pathrow").fetchall()
print(rows)

[(1, 26047)]


**If we want to change path_row attribute (column) of PathRow class:**

In [21]:
%%file /datos/example-django/myproj/satellital_images/models.py
from django.db import models

class Collection(models.Model):
    collection_type = models.CharField(unique=True, max_length=5)

class Sensor(models.Model):
    sensor_type = models.CharField(unique=True, max_length=10)
    collection_att = models.ManyToManyField("Collection")

class PathRow(models.Model):
    path_row_att = models.CharField(unique=True,max_length=10)#here the change
    
class Image(models.Model):
    download_url = models.CharField(max_length=200, default=None)
    sensor_image_att = models.ForeignKey(Sensor, related_name="sensor_images", on_delete=models.CASCADE)
    path_row_image_att = models.ForeignKey(PathRow, related_name="pathrow_images", on_delete=models.CASCADE)
    collection_image_att = models.ForeignKey(Collection, related_name="collection_images", on_delete=models.CASCADE)
    acquisition_date = models.DateTimeField()
    

Overwriting /datos/example-django/myproj/satellital_images/models.py


**Make migrations and migrate to save changes and apply changes:**

In [22]:
%%bash

cd /datos/example-django/myproj

python3 manage.py makemigrations 

#python3 manage.py makemigrations satellital_images

Migrations for 'satellital_images':
  satellital_images/migrations/0002_auto_20210323_1611.py
    - Alter field path_row_att on pathrow


In [23]:
%%bash

cd /datos/example-django/myproj/

python3 manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, satellital_images, sessions
Running migrations:
  Applying satellital_images.0002_auto_20210323_1611... OK


**Another scene:**

In [24]:
scene_0 = {"acquisitionDate": "2013-03-09", 
           "startTime": "2013-03-09", 
           "endTime": "2013-03-09", 
           "lowerLeftCoordinate": {"latitude": 16.65451, "longitude": -100.89458}, 
           "upperLeftCoordinate": {"latitude": 18.37292, "longitude": -100.51363}, 
           "upperRightCoordinate": {"latitude": 18.03105, "longitude": -98.86481}, 
           "lowerRightCoordinate": {"latitude": 16.31592, "longitude": -99.26119}, 
           "spatialFootprint": {"type": "Polygon", "coordinates": [[[-100.89458, 16.65451], 
                                                                    [-99.26119, 16.31592], 
                                                                    [-98.86481, 18.03105], 
                                                                    [-100.51363, 18.37292], 
                                                                    [-100.89458, 16.65451]]]}, 
           "sceneBounds": "-100.89458,16.31592,-98.86481,18.37292", 
           "browseUrl": "https://ims.cr.usgs.gov/browse/landsat_8_c1/2013/026/048/LT08_L1GT_026048_20130309_20170505_01_T2.jpg",
           "dataAccessUrl": "https://earthexplorer.usgs.gov/metadata/full/landsat_8_c1/LT80260482013068LGN02/",
           "downloadUrl": "https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260482013068LGN02/EE/",
           "entityId": "LT80260482013068LGN02",
           "displayId": "LT08_L1GT_026048_20130309_20170505_01_T2",
           "metadataUrl": "https://earthexplorer.usgs.gov/metadata/full/landsat_8_c1/LT80260482013068LGN02/?responseType=viewXml",
           "fgdcMetadataUrl": "https://earthexplorer.usgs.gov/metadata/fgdc/landsat_8_c1/LT80260482013068LGN02/",
           "modifiedDate": "2013-03-09 17:01:29",
           "orderUrl": "null",
           "bulkOrdered": "false",
           "ordered": "false",
           "summary": ""}

In [25]:
download_url = scene_0["downloadUrl"]
d = scene_0["displayId"]
list_d = d.split("_")
sensor=list_d[0]
path_row = list_d[2]
collection=list_d[5]
acq_date = scene_0["acquisitionDate"]

In [26]:
path_row

'026048'

In [27]:
collection

'01'

In [28]:
sensor

'LT08'

In [29]:
coll = Collection.objects.get_or_create(collection_type=collection)
pr = PathRow.objects.get_or_create(path_row_att=path_row)
sen = Sensor.objects.get_or_create(sensor_type=sensor)

In [30]:
coll

(<Collection: Collection object (1)>, False)

In [31]:
pr

(<PathRow: PathRow object (2)>, True)

In [32]:
sen

(<Sensor: Sensor object (1)>, False)

**True was created a new row, False just get the row**

In [33]:
coll = coll[0]
pr = pr[0]
sen = sen[0]

In [34]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
coll.save()
pr.save()
sen.save()
sen.collection_att.add(coll)

In [35]:
img = Image(download_url=download_url,
            sensor_image_att=sen,
            path_row_image_att=pr,
            collection_image_att=coll,
            acquisition_date=acq_date)

In [36]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
img.save()

/home/miuser/.local/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1370: RuntimeWarning: DateTimeField Image.acquisition_date received a naive datetime (2013-03-09 00:00:00) while time zone support is active.
  RuntimeWarning)


In [37]:
connection = sqlite3.connect("db.sqlite3")
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_user',), ('django_session',), ('satellital_images_collection',), ('satellital_images_sensor',), ('satellital_images_sensor_collection_att',), ('satellital_images_image',), ('satellital_images_pathrow',)]


**Next just to check, normally is not necessary to execute:**

In [38]:
rows = cursor.execute("SELECT * FROM satellital_images_image").fetchall()
print(rows)

[(1, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260472013068LGN02/EE/', '2013-03-09 00:00:00', 1, 1, 1), (2, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260482013068LGN02/EE/', '2013-03-09 00:00:00', 1, 2, 1)]


In [39]:
rows = cursor.execute("SELECT * FROM satellital_images_collection").fetchall()
print(rows)

[(1, '01')]


In [40]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor").fetchall()
print(rows)

[(1, 'LT08')]


In [41]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor_collection_att").fetchall()
print(rows)

[(1, 1, 1)]


In [42]:
rows = cursor.execute("SELECT * FROM satellital_images_pathrow").fetchall()
print(rows)

[(1, '26047'), (2, '26048')]


**Some queries:**

**Get `download_url`, `path_row_att`, `acquisition_date` of images related to `LT08` sensor**

In [43]:
Sensor.objects.filter(sensor_type="LT08")

<QuerySet [<Sensor: Sensor object (1)>]>

In [44]:
qs = Sensor.objects.filter(sensor_type="LT08")

In [45]:
r = qs[0]

In [46]:
r.sensor_images.distinct()

<QuerySet [<Image: Image object (1)>, <Image: Image object (2)>]>

In [47]:
qs = r.sensor_images.distinct()

In [49]:
for im in qs:
    print(im.download_url)
    print(im.path_row_image_att.path_row_att)
    print(im.acquisition_date)

https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260472013068LGN02/EE/
26047
2013-03-09 00:00:00+00:00
https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260482013068LGN02/EE/
26048
2013-03-09 00:00:00+00:00


**Access some `Collection` objects related to `Sensor` or `Image`:**

In [51]:
Sensor.collection_att.through.objects.all()

<QuerySet [<Sensor_collection_att: Sensor_collection_att object (1)>]>

In [54]:
r = Sensor.collection_att.through.objects.all()[0]

In [55]:
r.collection_id

1

In [57]:
r.collection.collection_images.distinct()

<QuerySet [<Image: Image object (1)>, <Image: Image object (2)>]>

In [58]:
r.collection.collection_type

'01'

**Filter**

In [59]:
Sensor.collection_att.through.objects.filter(collection="01")

<QuerySet [<Sensor_collection_att: Sensor_collection_att object (1)>]>

In [60]:
r = Sensor.collection_att.through.objects.filter(collection="01")[0]

In [62]:
r.collection.collection_images.distinct()

<QuerySet [<Image: Image object (1)>, <Image: Image object (2)>]>

In [64]:
r.collection.collection_type

'01'

**Another example for filter:**

In [66]:
Sensor.collection_att.through.objects.filter(sensor__sensor_type="LT08")

<QuerySet [<Sensor_collection_att: Sensor_collection_att object (1)>]>

In [67]:
r = Sensor.collection_att.through.objects.filter(sensor__sensor_type="LT08")[0]

In [68]:
r

<Sensor_collection_att: Sensor_collection_att object (1)>

In [70]:
r.sensor

<Sensor: Sensor object (1)>

In [71]:
r.sensor.sensor_type

'LT08'

In [72]:
r.sensor.sensor_images.distinct()

<QuerySet [<Image: Image object (1)>, <Image: Image object (2)>]>

In [73]:
r.sensor_id

1

In [74]:
r.collection.collection_images.distinct()

<QuerySet [<Image: Image object (1)>, <Image: Image object (2)>]>

**Get `download_url`, `path_row_att`, `acquisition_date` of images related to `LT08` sensor from many to many relation**

In [75]:
qs = r.collection.collection_images.distinct()

In [76]:
for im in qs:
    print(im.download_url)
    print(im.path_row_image_att.path_row_att)
    print(im.acquisition_date)

https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260472013068LGN02/EE/
26047
2013-03-09 00:00:00+00:00
https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80260482013068LGN02/EE/
26048
2013-03-09 00:00:00+00:00


# For data:

Using: https://github.com/loicdtx/lsru/

In [ ]:
!pip install lsru

In [1]:
from lsru import Usgs
import datetime

In [54]:
# Instantiate Usgs class and login
usgs = Usgs()
usgs.login()

True

In [7]:
[usgs.get_collection_name(k) for k in [4, 5, 7, 8]]

['LANDSAT_TM_C1', 'LANDSAT_TM_C1', 'LANDSAT_ETM_C1', 'LANDSAT_8_C1']

In [19]:
# Define query extent
#long, lat, long, lat
bbox = (-119.0, 14.0, -84.0, 33.0)

In [59]:
# Query the Usgs api to find scene intersecting with the spatio-temporal window
scene_list = usgs.search(collection='LANDSAT_8_C1',
                         bbox=bbox,
                         begin=datetime.datetime(2013,1,1),
                         end=datetime.datetime(2016,1,1),
                         max_results=10,
                         max_cloud_cover=40)


In [60]:
import json

In [61]:
i = 0
for i,scene in enumerate(scene_list):
    with open("scene" + str(i) + ".txt", 'w') as f:
        json.dump(scene, f)
        i+=1

In [63]:
# Query the Usgs api to find scene intersecting with the spatio-temporal window
scene_list2 = usgs.search(collection='LANDSAT_ETM_C1',
                         bbox=bbox,
                         begin=datetime.datetime(2013,1,1),
                         end=datetime.datetime(2016,1,1),
                         max_results=5,
                         max_cloud_cover=40)


In [70]:
for i,scene in enumerate(scene_list2,start=i):
    with open("/datos/example-django/scene" + str(i) + ".txt", 'w') as f:
        json.dump(scene, f)
        i+=1

10
11
12
13
14
